# MNIST with Convolutional Networks BN

In [103]:
import tensorflow as tf
from tensorflow.python import control_flow_ops
import numpy as np
import time, os
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [104]:
# Architecture
n_hidden_1 = 256
n_hidden_2 = 256

In [105]:
# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1

In [106]:
def conv_batch_norm(x, n_out, phase_train):
    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)

    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
    mean, var = control_flow_ops.cond(phase_train,
        mean_var_with_update,
        lambda: (ema_mean, ema_var))

    normed = tf.nn.batch_norm_with_global_normalization(x, mean, var,
        beta, gamma, 1e-3, True)
    return normed

In [107]:
def layer_batch_norm(x, n_out, phase_train):
    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)

    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
    mean, var = control_flow_ops.cond(phase_train,
        mean_var_with_update,
        lambda: (ema_mean, ema_var))

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var,
        beta, gamma, 1e-3, True)
    return tf.reshape(normed, [-1, n_out])

In [108]:
def filter_summary(V, weight_shape):
    ix = weight_shape[0]
    iy = weight_shape[1]
    cx, cy = 8, 8
    V_T = tf.transpose(V, (3, 0, 1, 2))
    tf.summary.image("filters", V_T, max_outputs=64)

In [109]:
def conv2d(input, weight_shape, bias_shape, phase_train, visualize=False):
    incoming = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(stddev=(2.0/incoming)**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    if visualize:
        filter_summary(W, weight_shape)
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    logits = tf.nn.bias_add(tf.nn.conv2d(input, W, strides=[1, 1, 1, 1], padding='SAME'), b)
    return tf.nn.relu(conv_batch_norm(logits, weight_shape[3], phase_train))

In [110]:
def max_pool(input, k=2):
    return tf.nn.max_pool(input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [111]:
def layer(input, weight_shape, bias_shape, phase_train):
    weight_stddev = (2.0/weight_shape[0])**0.5
    w_init = tf.random_normal_initializer(stddev=weight_stddev)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", weight_shape, initializer=w_init)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    logits = tf.matmul(input, W) + b
    return tf.nn.relu(layer_batch_norm(logits, weight_shape[1], phase_train))

In [112]:
def inference(x, keep_prob, phase_train):

    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    with tf.variable_scope("conv_1"):
        conv_1 = conv2d(x, [5, 5, 1, 32], [32], phase_train, visualize=True)
        pool_1 = max_pool(conv_1)

    with tf.variable_scope("conv_2"):
        conv_2 = conv2d(pool_1, [5, 5, 32, 64], [64], phase_train)
        pool_2 = max_pool(conv_2)

    with tf.variable_scope("fc"):
        pool_2_flat = tf.reshape(pool_2, [-1, 7 * 7 * 64])
        fc_1 = layer(pool_2_flat, [7*7*64, 1024], [1024], phase_train)
        
        # apply dropout
        fc_1_drop = tf.nn.dropout(fc_1, keep_prob)

    with tf.variable_scope("output"):
        output = layer(fc_1_drop, [1024, 10], [10], phase_train)

    return output

In [113]:
def loss(output, y):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y)
    loss = tf.reduce_mean(xentropy)    
    return loss

In [114]:
def training(cost, global_step):
    tf.summary.scalar("cost", cost)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [115]:
def evaluate(output, y):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("validation error", (1.0 - accuracy))
    return accuracy

In [116]:
if __name__ == '__main__':

    with tf.device("/gpu:0"):

        with tf.Graph().as_default():

            with tf.variable_scope("mnist_conv_bn_model"):

                x = tf.placeholder("float", [None, 784]) # mnist data image of shape 28*28=784
                y = tf.placeholder("float", [None, 10]) # 0-9 digits recognition => 10 classes
                keep_prob = tf.placeholder(tf.float32) # dropout probability
                phase_train = tf.placeholder(tf.bool) # training or testing

                output = inference(x, keep_prob, phase_train)

                cost = loss(output, y)

                global_step = tf.Variable(0, name='global_step', trainable=False)

                train_op = training(cost, global_step)

                eval_op = evaluate(output, y)

                summary_op = tf.summary.merge_all()

                saver = tf.train.Saver()

                sess = tf.Session()

                summary_writer = tf.summary.FileWriter("conv_mnist_bn_logs/",
                                                    graph_def=sess.graph_def)

                
                init_op = tf.global_variables_initializer()

                sess.run(init_op)


                # Training cycle
                for epoch in range(training_epochs):
                    
                    avg_cost = 0.
                    total_batch = int(mnist.train.num_examples/batch_size)
                    # Loop over all batches
                    for i in range(total_batch):
                        # Fit training using batch data
                        minibatch_x, minibatch_y = mnist.train.next_batch(batch_size)                        
                        _, new_cost = sess.run([train_op, cost], feed_dict={x: minibatch_x, y: minibatch_y, keep_prob: 1, phase_train: True})
                        # Compute average loss                
                        avg_cost += new_cost/total_batch
                        #print("Epoch %d, minibatch %d of %d. Cost = %0.4f." %(epoch, i, total_batch, new_cost))
                        
                    # Display logs per epoch step
                    if epoch % display_step == 0:
                        print("Epoch:", '%04d' % (epoch), "cost =", "{:.9f}".format(avg_cost))

                        accuracy = sess.run(eval_op, feed_dict={x: mnist.validation.images, y: mnist.validation.labels, keep_prob: 1, phase_train: False})

                        print("Validation Error:", (1 - accuracy))

                        summary_str = sess.run(summary_op, feed_dict={x: minibatch_x, y: minibatch_y, keep_prob: 1, phase_train: False})
                        summary_writer.add_summary(summary_str, sess.run(global_step))

                        saver.save(sess, "conv_mnist_bn_logs/model-checkpoint", global_step=global_step)


                print("Optimization Finished!")


                accuracy = sess.run(eval_op, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1, phase_train: False})

                print("Test Accuracy:", accuracy)

INFO:tensorflow:Summary name validation error is illegal; using validation_error instead.
Epoch 0, minibatch 0 of 550. Cost = 2.4926.
Epoch 0, minibatch 1 of 550. Cost = 1.4097.
Epoch 0, minibatch 2 of 550. Cost = 1.2033.
Epoch 0, minibatch 3 of 550. Cost = 0.9574.
Epoch 0, minibatch 4 of 550. Cost = 0.9680.
Epoch 0, minibatch 5 of 550. Cost = 0.9249.
Epoch 0, minibatch 6 of 550. Cost = 0.9259.
Epoch 0, minibatch 7 of 550. Cost = 0.8236.
Epoch 0, minibatch 8 of 550. Cost = 0.7762.
Epoch 0, minibatch 9 of 550. Cost = 0.7670.
Epoch 0, minibatch 10 of 550. Cost = 0.6926.
Epoch 0, minibatch 11 of 550. Cost = 0.6235.
Epoch 0, minibatch 12 of 550. Cost = 0.6695.
Epoch 0, minibatch 13 of 550. Cost = 0.6211.
Epoch 0, minibatch 14 of 550. Cost = 0.6053.
Epoch 0, minibatch 15 of 550. Cost = 0.6999.
Epoch 0, minibatch 16 of 550. Cost = 0.6078.
Epoch 0, minibatch 17 of 550. Cost = 0.5543.
Epoch 0, minibatch 18 of 550. Cost = 0.5103.
Epoch 0, minibatch 19 of 550. Cost = 0.6031.
Epoch 0, minibatch 2

Epoch 0, minibatch 176 of 550. Cost = 0.1571.
Epoch 0, minibatch 177 of 550. Cost = 0.0946.
Epoch 0, minibatch 178 of 550. Cost = 0.1001.
Epoch 0, minibatch 179 of 550. Cost = 0.1393.
Epoch 0, minibatch 180 of 550. Cost = 0.1329.
Epoch 0, minibatch 181 of 550. Cost = 0.1333.
Epoch 0, minibatch 182 of 550. Cost = 0.1534.
Epoch 0, minibatch 183 of 550. Cost = 0.1187.
Epoch 0, minibatch 184 of 550. Cost = 0.1098.
Epoch 0, minibatch 185 of 550. Cost = 0.1275.
Epoch 0, minibatch 186 of 550. Cost = 0.1069.
Epoch 0, minibatch 187 of 550. Cost = 0.1065.
Epoch 0, minibatch 188 of 550. Cost = 0.1678.
Epoch 0, minibatch 189 of 550. Cost = 0.1231.
Epoch 0, minibatch 190 of 550. Cost = 0.0766.
Epoch 0, minibatch 191 of 550. Cost = 0.1018.
Epoch 0, minibatch 192 of 550. Cost = 0.0761.
Epoch 0, minibatch 193 of 550. Cost = 0.0608.
Epoch 0, minibatch 194 of 550. Cost = 0.0788.
Epoch 0, minibatch 195 of 550. Cost = 0.0789.
Epoch 0, minibatch 196 of 550. Cost = 0.0833.
Epoch 0, minibatch 197 of 550. Cos

Epoch 0, minibatch 359 of 550. Cost = 0.0548.
Epoch 0, minibatch 360 of 550. Cost = 0.0468.
Epoch 0, minibatch 361 of 550. Cost = 0.0889.
Epoch 0, minibatch 362 of 550. Cost = 0.0874.
Epoch 0, minibatch 363 of 550. Cost = 0.0328.
Epoch 0, minibatch 364 of 550. Cost = 0.0495.
Epoch 0, minibatch 365 of 550. Cost = 0.0453.
Epoch 0, minibatch 366 of 550. Cost = 0.1076.
Epoch 0, minibatch 367 of 550. Cost = 0.0787.
Epoch 0, minibatch 368 of 550. Cost = 0.0785.
Epoch 0, minibatch 369 of 550. Cost = 0.0676.
Epoch 0, minibatch 370 of 550. Cost = 0.0342.
Epoch 0, minibatch 371 of 550. Cost = 0.0915.
Epoch 0, minibatch 372 of 550. Cost = 0.1206.
Epoch 0, minibatch 373 of 550. Cost = 0.0829.
Epoch 0, minibatch 374 of 550. Cost = 0.0595.
Epoch 0, minibatch 375 of 550. Cost = 0.1443.
Epoch 0, minibatch 376 of 550. Cost = 0.0327.
Epoch 0, minibatch 377 of 550. Cost = 0.0409.
Epoch 0, minibatch 378 of 550. Cost = 0.1366.
Epoch 0, minibatch 379 of 550. Cost = 0.0832.
Epoch 0, minibatch 380 of 550. Cos

Epoch 0, minibatch 541 of 550. Cost = 0.0476.
Epoch 0, minibatch 542 of 550. Cost = 0.0587.
Epoch 0, minibatch 543 of 550. Cost = 0.0559.
Epoch 0, minibatch 544 of 550. Cost = 0.0953.
Epoch 0, minibatch 545 of 550. Cost = 0.0594.
Epoch 0, minibatch 546 of 550. Cost = 0.0183.
Epoch 0, minibatch 547 of 550. Cost = 0.0492.
Epoch 0, minibatch 548 of 550. Cost = 0.1178.
Epoch 0, minibatch 549 of 550. Cost = 0.1010.
Epoch: 0000 cost = 0.147250545
Validation Error: 0.010800004005432129
Epoch 1, minibatch 0 of 550. Cost = 0.0605.
Epoch 1, minibatch 1 of 550. Cost = 0.1173.
Epoch 1, minibatch 2 of 550. Cost = 0.0486.
Epoch 1, minibatch 3 of 550. Cost = 0.0349.
Epoch 1, minibatch 4 of 550. Cost = 0.0333.
Epoch 1, minibatch 5 of 550. Cost = 0.0301.
Epoch 1, minibatch 6 of 550. Cost = 0.0172.
Epoch 1, minibatch 7 of 550. Cost = 0.1175.
Epoch 1, minibatch 8 of 550. Cost = 0.0314.
Epoch 1, minibatch 9 of 550. Cost = 0.0542.
Epoch 1, minibatch 10 of 550. Cost = 0.0696.
Epoch 1, minibatch 11 of 550. C

Epoch 1, minibatch 173 of 550. Cost = 0.0281.
Epoch 1, minibatch 174 of 550. Cost = 0.0171.
Epoch 1, minibatch 175 of 550. Cost = 0.0632.
Epoch 1, minibatch 176 of 550. Cost = 0.0358.
Epoch 1, minibatch 177 of 550. Cost = 0.0497.
Epoch 1, minibatch 178 of 550. Cost = 0.0240.
Epoch 1, minibatch 179 of 550. Cost = 0.0319.
Epoch 1, minibatch 180 of 550. Cost = 0.0585.
Epoch 1, minibatch 181 of 550. Cost = 0.0303.
Epoch 1, minibatch 182 of 550. Cost = 0.0416.
Epoch 1, minibatch 183 of 550. Cost = 0.0398.
Epoch 1, minibatch 184 of 550. Cost = 0.0807.
Epoch 1, minibatch 185 of 550. Cost = 0.0100.
Epoch 1, minibatch 186 of 550. Cost = 0.0164.
Epoch 1, minibatch 187 of 550. Cost = 0.0491.
Epoch 1, minibatch 188 of 550. Cost = 0.0698.
Epoch 1, minibatch 189 of 550. Cost = 0.0470.
Epoch 1, minibatch 190 of 550. Cost = 0.0627.
Epoch 1, minibatch 191 of 550. Cost = 0.0415.
Epoch 1, minibatch 192 of 550. Cost = 0.0139.
Epoch 1, minibatch 193 of 550. Cost = 0.0227.
Epoch 1, minibatch 194 of 550. Cos

...

Epoch 8, minibatch 310 of 550. Cost = 0.0022.
Epoch 8, minibatch 311 of 550. Cost = 0.0050.
Epoch 8, minibatch 312 of 550. Cost = 0.0031.
Epoch 8, minibatch 313 of 550. Cost = 0.0049.
Epoch 8, minibatch 314 of 550. Cost = 0.0076.
Epoch 8, minibatch 315 of 550. Cost = 0.0093.
Epoch 8, minibatch 316 of 550. Cost = 0.0012.
Epoch 8, minibatch 317 of 550. Cost = 0.0017.
Epoch 8, minibatch 318 of 550. Cost = 0.0008.
Epoch 8, minibatch 319 of 550. Cost = 0.0023.
Epoch 8, minibatch 320 of 550. Cost = 0.0143.
Epoch 8, minibatch 321 of 550. Cost = 0.0030.
Epoch 8, minibatch 322 of 550. Cost = 0.0018.
Epoch 8, minibatch 323 of 550. Cost = 0.0053.
Epoch 8, minibatch 324 of 550. Cost = 0.0009.
Epoch 8, minibatch 325 of 550. Cost = 0.0019.
Epoch 8, minibatch 326 of 550. Cost = 0.0016.
Epoch 8, minibatch 327 of 550. Cost = 0.0013.
Epoch 8, minibatch 328 of 550. Cost = 0.0045.
Epoch 8, minibatch 329 of 550. Cost = 0.0040.
Epoch 8, minibatch 330 of 550. Cost = 0.0500.
Epoch 8, minibatch 331 of 550. Cos

Epoch 8, minibatch 493 of 550. Cost = 0.0085.
Epoch 8, minibatch 494 of 550. Cost = 0.0079.
Epoch 8, minibatch 495 of 550. Cost = 0.0214.
Epoch 8, minibatch 496 of 550. Cost = 0.0104.
Epoch 8, minibatch 497 of 550. Cost = 0.0139.
Epoch 8, minibatch 498 of 550. Cost = 0.0563.
Epoch 8, minibatch 499 of 550. Cost = 0.0019.
Epoch 8, minibatch 500 of 550. Cost = 0.0039.
Epoch 8, minibatch 501 of 550. Cost = 0.0049.
Epoch 8, minibatch 502 of 550. Cost = 0.0229.
Epoch 8, minibatch 503 of 550. Cost = 0.0047.
Epoch 8, minibatch 504 of 550. Cost = 0.0003.
Epoch 8, minibatch 505 of 550. Cost = 0.0143.
Epoch 8, minibatch 506 of 550. Cost = 0.0129.
Epoch 8, minibatch 507 of 550. Cost = 0.0008.
Epoch 8, minibatch 508 of 550. Cost = 0.0238.
Epoch 8, minibatch 509 of 550. Cost = 0.0117.
Epoch 8, minibatch 510 of 550. Cost = 0.0009.
Epoch 8, minibatch 511 of 550. Cost = 0.0020.
Epoch 8, minibatch 512 of 550. Cost = 0.0015.
Epoch 8, minibatch 513 of 550. Cost = 0.0018.
Epoch 8, minibatch 514 of 550. Cos

Epoch 9, minibatch 126 of 550. Cost = 0.0018.
Epoch 9, minibatch 127 of 550. Cost = 0.0180.
Epoch 9, minibatch 128 of 550. Cost = 0.0005.
Epoch 9, minibatch 129 of 550. Cost = 0.0109.
Epoch 9, minibatch 130 of 550. Cost = 0.0021.
Epoch 9, minibatch 131 of 550. Cost = 0.0052.
Epoch 9, minibatch 132 of 550. Cost = 0.0207.
Epoch 9, minibatch 133 of 550. Cost = 0.0038.
Epoch 9, minibatch 134 of 550. Cost = 0.0060.
Epoch 9, minibatch 135 of 550. Cost = 0.0006.
Epoch 9, minibatch 136 of 550. Cost = 0.0017.
Epoch 9, minibatch 137 of 550. Cost = 0.0058.
Epoch 9, minibatch 138 of 550. Cost = 0.0014.
Epoch 9, minibatch 139 of 550. Cost = 0.0101.
Epoch 9, minibatch 140 of 550. Cost = 0.0008.
Epoch 9, minibatch 141 of 550. Cost = 0.0022.
Epoch 9, minibatch 142 of 550. Cost = 0.0016.
Epoch 9, minibatch 143 of 550. Cost = 0.0322.
Epoch 9, minibatch 144 of 550. Cost = 0.0008.
Epoch 9, minibatch 145 of 550. Cost = 0.0013.
Epoch 9, minibatch 146 of 550. Cost = 0.0292.
Epoch 9, minibatch 147 of 550. Cos

Epoch 9, minibatch 306 of 550. Cost = 0.0135.
Epoch 9, minibatch 307 of 550. Cost = 0.0188.
Epoch 9, minibatch 308 of 550. Cost = 0.0013.
Epoch 9, minibatch 309 of 550. Cost = 0.0019.
Epoch 9, minibatch 310 of 550. Cost = 0.0264.
Epoch 9, minibatch 311 of 550. Cost = 0.0014.
Epoch 9, minibatch 312 of 550. Cost = 0.0027.
Epoch 9, minibatch 313 of 550. Cost = 0.0018.
Epoch 9, minibatch 314 of 550. Cost = 0.0035.
Epoch 9, minibatch 315 of 550. Cost = 0.0019.
Epoch 9, minibatch 316 of 550. Cost = 0.0025.
Epoch 9, minibatch 317 of 550. Cost = 0.0035.
Epoch 9, minibatch 318 of 550. Cost = 0.0040.
Epoch 9, minibatch 319 of 550. Cost = 0.0127.
Epoch 9, minibatch 320 of 550. Cost = 0.0046.
Epoch 9, minibatch 321 of 550. Cost = 0.0034.
Epoch 9, minibatch 322 of 550. Cost = 0.0087.
Epoch 9, minibatch 323 of 550. Cost = 0.0027.
Epoch 9, minibatch 324 of 550. Cost = 0.0008.
Epoch 9, minibatch 325 of 550. Cost = 0.0164.
Epoch 9, minibatch 326 of 550. Cost = 0.0094.
Epoch 9, minibatch 327 of 550. Cos

Epoch 9, minibatch 485 of 550. Cost = 0.0021.
Epoch 9, minibatch 486 of 550. Cost = 0.0016.
Epoch 9, minibatch 487 of 550. Cost = 0.0126.
Epoch 9, minibatch 488 of 550. Cost = 0.0003.
Epoch 9, minibatch 489 of 550. Cost = 0.0137.
Epoch 9, minibatch 490 of 550. Cost = 0.0225.
Epoch 9, minibatch 491 of 550. Cost = 0.0112.
Epoch 9, minibatch 492 of 550. Cost = 0.0015.
Epoch 9, minibatch 493 of 550. Cost = 0.0110.
Epoch 9, minibatch 494 of 550. Cost = 0.0226.
Epoch 9, minibatch 495 of 550. Cost = 0.0105.
Epoch 9, minibatch 496 of 550. Cost = 0.0023.
Epoch 9, minibatch 497 of 550. Cost = 0.0061.
Epoch 9, minibatch 498 of 550. Cost = 0.0273.
Epoch 9, minibatch 499 of 550. Cost = 0.0202.
Epoch 9, minibatch 500 of 550. Cost = 0.0037.
Epoch 9, minibatch 501 of 550. Cost = 0.0069.
Epoch 9, minibatch 502 of 550. Cost = 0.0038.
Epoch 9, minibatch 503 of 550. Cost = 0.0166.
Epoch 9, minibatch 504 of 550. Cost = 0.0009.
Epoch 9, minibatch 505 of 550. Cost = 0.0364.
Epoch 9, minibatch 506 of 550. Cos